In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

In [2]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'
response = requests.get(URL)

In [3]:
type(response)

requests.models.Response

In [4]:
response.status_code

200

In [5]:
soup = BS(response.text)

**Tennessee District 07**

In [6]:
r = requests.get('https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N')
soup = BS(r.text,'html.parser')

In [7]:
#Find the district (Ex. district 07)
district = soup.find('h1',attrs = {'class':'Hero-title'})
district.text

'Tennessee District 07 2020 Race'

In [8]:
#Find the district number (Ex. district 07)
district = soup.find('h1',attrs = {'class':'Hero-title'})
dist_num = str(re.findall(r"0\d{1}\W", district.text)).replace(" ", "").replace("[", "").replace("]", "").replace("'", "")
dist_num

'07'

In [9]:
#Find member blocks for each candidate
members = soup.find_all('div',attrs = {'class':'Members--list-item'})

In [10]:
#Fake list Candidates, grabs each member column with members--bio u-richtext
Candidates = []
for cand in members:
    Candidates.append(cand.find('div', attrs = {'class' : 'Members--bio u-richtext'}))

In [11]:
#From Candidates, extract just the text parts inclding the candidate name and header
Cand_list_text = []
for i in Candidates:
    Cand_list_text.append(i.get_text(strip=True))
Cand_list_text

['Mark Green (R) •Incumbent•Winner(69.9% of vote)',
 'Kiran Sreepada (D)(27.3% of vote)',
 'Ronald Brown (I)(2.2% of vote)',
 'Scott Vieira Jr (I)(0.6% of vote)']

In [12]:
CandNames = []              #Make empty lists
district_num = []
incumbent=[]
winner=[] 

for i in Cand_list_text:    #add information about candidate to lists
    winner.append(str(re.findall(r"Winner", i)).replace("[", "").replace("]", "").replace("'", ""))
    incumbent.append(str(re.findall(r"Incumbent", i)).replace("[", "").replace("]", "").replace("'", ""))
    CandNames.append(str(re.findall(r".+\([A-Z]", i)).replace("'", "").replace("[", "").replace("]", "").replace("(R", "").replace("(D", "").replace("(I", ""))
    district_num.append(str(re.findall(r"0\d{1}\W", district.text)).replace(" ", "").replace("[", "").replace("]", "").replace("'", ""))
    

In [13]:
#Make a dataframe using dictionary
di = {'Candidate Name': CandNames,'District Number':district_num,  'Incumbent':incumbent, 'Winner':winner}
pd.DataFrame(di)

,Candidate Name,District Number,Incumbent,Winner
0,Mark Green,07,Incumbent,Winner
1,Kiran Sreepada,07,,
2,Ronald Brown,07,,
3,Scott Vieira Jr,07,,


# All TN Districts

In [14]:
#testing url
d_num = 10
baseurl = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN0'  + str(d_num) + '&spec=N'
r = requests.get('https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N')
soup = BS(r.text,'html.parser')
print(baseurl)

https://www.opensecrets.org/races/candidates?cycle=2020&id=TN010&spec=N


In [15]:
d = soup.find('ol',attrs = {'class':'SubNav-items'})  #Navigation showing list of districts
a_tags = d.find_all('a')

In [16]:
a_tags = soup.find('ol',attrs = {'class':'SubNav-items'}).find_all('a')  #Navigation showing list of districts
a_tags

[<a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TNS2&amp;spec=N">
                         Tennessee Senate <i aria-hidden="true" class="fa fa-angle-right"></i>
 </a>,
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN01&amp;spec=N">
                         Tennessee District 01 <i aria-hidden="true" class="fa fa-angle-right"></i>
 </a>,
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN02&amp;spec=N">
                         Tennessee District 02 <i aria-hidden="true" class="fa fa-angle-right"></i>
 </a>,
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN03&amp;spec=N">
                         Tennessee District 03 <i aria-hidden="true" class="fa fa-angle-right"></i>
 </a>,
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN04&amp;spec=N">
                         Tennessee District 04 <i aria-hidden="true" class="fa fa-angle-right"></i>
 </a>,
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN

In [17]:
d_count = 0                               #Start counting districts in state
for i in a_tags:
    if str.isdigit(i.text.strip()[-1]):   #if the last substring is a digit
        d_count+=1                        #increase district count by 1
    #print(i.text.strip()[-1])
    
d_count  

9

In [18]:
count = 1

district_num = []
incumbent=[]
winner=[]
CandNames = []

while count <= d_count:     #for each district number
    url = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN0'  + str(count) + '&spec=N'   #here's the url
    r = requests.get(url)
    soup = BS(r.text,'html.parser')
    
    print(url)

    district = soup.find('h1',attrs = {'class':'Hero-title'})
    dist_num = str(re.findall(r"0\d{1}\W", district.text)).replace(" ", "").replace("[", "").replace("]", "").replace("'", "")

    #Find member blocks for each candidate
    members = soup.find_all('div',attrs = {'class':'Members--list-item'})

    #make list Candidates, grabs each member column with members--bio u-richtext
    Candidates = []
    for cand in members:
        Candidates.append(cand.find('div', attrs = {'class' : 'Members--bio u-richtext'}))
    
    
    #from Candidates, extract just the text parts inclding the candidate name and header
    Cand_list_text = []
    for i in Candidates:
        Cand_list_text.append(i.get_text(strip=True))
    
    
    for i in Cand_list_text:
        winner.append(str(re.findall(r"Winner", i)).replace("[", "").replace("]", "").replace("'", ""))
        incumbent.append(str(re.findall(r"Incumbent", i)).replace("[", "").replace("]", "").replace("'", ""))
        CandNames.append(str(re.findall(r".+\([A-Z]", i)).replace("'", "").replace("[", "").replace("]", "").replace("(R", "").replace("(D", "").replace("(I", ""))
        district_num.append(str(re.findall(r"0\d{1}\W", district.text)).replace(" ", "").replace("[", "").replace("]", "").replace("'", ""))
    
    count+=1 # increment count

https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N
https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N


In [19]:
#Make a dataframe using dictionary
di = {'Candidate Name': CandNames,'District Number':district_num,  'Incumbent':incumbent, 'Winner':winner}
z = pd.DataFrame(di)

In [20]:
z.to_csv('test.csv')